In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)


import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp
from sklearn.linear_model import LinearRegression

In [2]:


         
dataset = dp.Dataset(pd.read_excel("../datasets/2_class.xls"), name="2_class.xls")

def detect_anomalies(column):
    X = np.arange(len(column)).reshape(-1, 1)  # Используем индекс как признак
    y = column.values.reshape(-1, 1)
    
    model = LinearRegression().fit(X, y)
    pred = model.predict(X)
    residuals = np.abs(y - pred).flatten()
    
    # Определяем аномалии через IQR
    Q1 = np.percentile(residuals, 25)
    Q3 = np.percentile(residuals, 75)
    IQR = Q3 - Q1
    threshold = Q3 + 1.5 * IQR
    
    return (residuals > threshold).astype(int)

    

In [3]:
# Копируем исходный датасет и удаляем 'Class'
new_df = dataset.drop('Class', axis=1).copy()

# Для каждого столбца применяем функцию detect_anomalies
for col in new_df.columns:
    new_df[col] = detect_anomalies(new_df[col])

# Возвращаем столбец 'Class' без изменений
new_df['Class'] = dataset['Class']

# Выводим результат
print(new_df.head())
new_df.describe()

   Ubs,V  Ibs,A  Isun,A  Ipt1,A  Ipt2,A  Ipt3,A  Ipt4,A  Ipt5,A  Ipt6,A  \
0      0      0       0       0       0       0       0       0       0   
1      0      0       0       0       0       0       0       0       0   
2      0      0       0       0       0       0       0       0       0   
3      0      0       0       0       0       0       0       0       0   
4      0      0       0       0       0       0       0       0       0   

   Ipt7,A  ...  TDS7,C  TDS8,C  TDS9,C  TKpt,C  TGbv,C  TNap,C  TPrd2,C  \
0       0  ...       0       0       0       0       0       0        0   
1       0  ...       0       0       0       0       0       0        0   
2       0  ...       0       0       0       0       0       0        0   
3       0  ...       0       0       0       0       0       0        0   
4       0  ...       0       0       0       0       0       0        0   

   TPrd1,C  TDS24,C  Class  
0        0        0      0  
1        0        0      0  
2        0 

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C",Class
count,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,...,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000,2679.000
mean,0.073,0.069,0.066,0.072,0.072,0.062,0.062,0.063,0.062,0.064,...,0.073,0.077,0.074,0.075,0.077,0.078,0.074,0.067,0.067,0.121
std,0.260,0.254,0.249,0.258,0.259,0.242,0.242,0.242,0.241,0.244,...,0.260,0.266,0.262,0.263,0.267,0.269,0.262,0.250,0.250,0.326
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [6]:
new_df.to_excel("../datasets/anomalies_dataset.xlsx", index=False)  # index=False убирает запись индексов